In [ ]:
import torch
import torchvision
import numpy as np
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [ ]:
torch.cuda.is_available()

True

In [ ]:
!git clone https://github.com/ContinualAI/colab.git continualai/colab
from continualai.colab.scripts import mnist
mnist.init()

Cloning into 'continualai/colab'...
remote: Enumerating objects: 378, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 378 (delta 78), reused 64 (delta 62), pack-reused 258
Receiving objects: 100% (378/378), 26.97 MiB | 19.23 MiB/s, done.
Resolving deltas: 100% (198/198), done.
Download complete.
Save complete.


100%|██████████| 9912422/9912422 [00:00<00:00, 35802210.91it/s]


Extracting data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to data/mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 893871.62it/s]


Extracting data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to data/mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 9151539.21it/s]


Extracting data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to data/mnist/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 11694615.57it/s]


Extracting data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/mnist/MNIST/raw



In [ ]:
use_cuda = True

use_cuda = use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu");
torch.manual_seed(1);

In [ ]:
x_train, t_train, x_test, t_test = mnist.load()

In [ ]:
def permute_mnist(mnist, seed):
    np.random.seed(seed)
    print("starting permutation...")
    h = w = 28
    perm_inds = list(range(h*w))
    np.random.shuffle(perm_inds)
    # print(perm_inds)
    perm_mnist = []
    for set in mnist:
        num_img = set.shape[0]
        flat_set = set.reshape(num_img, w * h)
        perm_mnist.append(flat_set[:, perm_inds].reshape(num_img, 1, w, h))
    print("done.")
    return perm_mnist

In [ ]:
# task 1
task_1 = [(x_train, t_train), (x_test, t_test)]

# task 2
x_train2, x_test2 = permute_mnist([x_train, x_test], 1)
task_2 = [(x_train2, t_train), (x_test2, t_test)]

# task 3
x_train3, x_test3 = permute_mnist([x_train, x_test], 2)
task_3 = [(x_train3, t_train), (x_test3, t_test)]

# task list

# task 4
x_train4, x_test4 = permute_mnist([x_train, x_test], 3)
task_4 = [(x_train4, t_train), (x_test4, t_test)]

# task 3
x_train5, x_test5 = permute_mnist([x_train, x_test], 4)
task_5 = [(x_train5, t_train), (x_test5, t_test)]

tasks = [task_1, task_2, task_3,task_4,task_5]

starting permutation...
done.
starting permutation...
done.
starting permutation...
done.
starting permutation...
done.


In [ ]:
"""VGG(
  (convs): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (16): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (18): ReLU()
    (19): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (20): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (21): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (22): ReLU()
    (23): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (24): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (25): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (26): ReLU()
    (27): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (28): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (29): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (30): ReLU()
    (31): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (32): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (33): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (34): ReLU()
    (35): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (36): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (37): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (38): ReLU()
    (39): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (40): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (41): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (42): ReLU()
    (43): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (44): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (45): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (46): ReLU()
    (47): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (48): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (49): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (50): ReLU()
    (51): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (fcs): Sequential(
    (0): Linear(in_features=512, out_features=4096, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=4096, out_features=4096, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=4096, out_features=1000, bias=True)
  )
)
"""



# VGG_NPF
class VGGNEW(nn.Module):
    def __init__(self):
        super().__init__()
        self.C11= nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.B11=nn.BatchNorm2d(64)


        self.C12=nn.Conv2d(64, 64, kernel_size=(3, 3),padding=1)
        self.B12=nn.BatchNorm2d(64)
        self.M12=nn.AvgPool2d(2,2)


        self.C21=nn.Conv2d(64, 128,kernel_size=(3, 3), padding=(1, 1))
        self.B21=nn.BatchNorm2d(128)

        self.C22=nn.Conv2d(128, 128, kernel_size=(3, 3), padding=(1, 1))
        self.B22=nn.BatchNorm2d(128)
        self.M22=nn.AvgPool2d(2,2,padding=1)

        self.C31=nn.Conv2d(128, 256, kernel_size=(3, 3), padding=(1, 1))
        self.B31=nn.BatchNorm2d(256)


        self.C32=nn.Conv2d(256, 256, kernel_size=(3, 3),  padding=(1, 1))
        self.B32=nn.BatchNorm2d(256)

        self.C33=nn.Conv2d(256, 256, kernel_size=(3, 3),  padding=(1, 1))
        self.B33=nn.BatchNorm2d(256)
        self.M33=nn.AvgPool2d(2,2,padding=1)

        self.C41=nn.Conv2d(256, 512, kernel_size=(3, 3),  padding=(1, 1))
        self.B41=nn.BatchNorm2d(512)

        self.C42=nn.Conv2d(512, 512, kernel_size=(3, 3),  padding=(1, 1))
        self.B42=nn.BatchNorm2d(512)


        self.C43=nn.Conv2d(512, 512, kernel_size=(3, 3),  padding=(1, 1))
        self.B43=nn.BatchNorm2d(512)
        self.M43=nn.AvgPool2d(2,2,padding=1)

        self.C51=nn.Conv2d(512, 512, kernel_size=(3, 3), padding=(1, 1))
        self.B51=nn.BatchNorm2d(512)

        self.C52=nn.Conv2d(512, 512, kernel_size=(3, 3),  padding=(1, 1))
        self.B52=nn.BatchNorm2d(512)

        self.C53=nn.Conv2d(512, 512, kernel_size=(3, 3), padding=(1, 1))
        self.B53=nn.BatchNorm2d(512)
        self.M53=nn.AvgPool2d(2,2,padding=1)


    def forward(self, xb):
        #out = xb.view(xb.size(0), -1)
        self.Gate_NPF=[]
        SIGMOID=torch.nn.Sigmoid()
        beta=10

        out = self.C11(xb)
        out=self.B11(out)
        self.Gate_NPF.append(SIGMOID(beta*out))


        out = self.C12(out)
        out=self.B12(out)
        self.Gate_NPF.append(SIGMOID(beta*out))
        out = self.M12(out)

        out = self.C21(out)
        out=self.B21(out)
        self.Gate_NPF.append(SIGMOID(beta*out))


        out = self.C22(out)
        out=self.B22(out)
        self.Gate_NPF.append(SIGMOID(beta*out))
        out = self.M22(out)

        out = self.C31(out)
        out=self.B31(out)
        self.Gate_NPF.append(SIGMOID(beta*out))


        out = self.C32(out)
        out=self.B32(out)
        self.Gate_NPF.append(SIGMOID(beta*out))


        out = self.C33(out)
        out=self.B33(out)
        self.Gate_NPF.append(SIGMOID(beta*out))
        out = self.M33(out)

        out = self.C41(out)
        out=self.B41(out)
        self.Gate_NPF.append(SIGMOID(beta*out))


        out = self.C42(out)
        out=self.B42(out)
        self.Gate_NPF.append(SIGMOID(beta*out))


        out = self.C43(out)
        out=self.B43(out)
        self.Gate_NPF.append(SIGMOID(beta*out))
        out = self.M43(out)

        out = self.C51(out)
        out=self.B51(out)
        self.Gate_NPF.append(SIGMOID(beta*out))


        out = self.C52(out)
        out=self.B52(out)
        self.Gate_NPF.append(SIGMOID(beta*out))


        out = self.C53(out)
        out=self.B53(out)
        self.Gate_NPF.append(SIGMOID(beta*out))
        out = self.M53(out)

    def Return_NPF_Gates(self):
      return self.Gate_NPF

In [ ]:
class VGGWeight(nn.Module):
    def __init__(self):
        super().__init__()
        self.C11= nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.B11=nn.BatchNorm2d(64)


        self.C12=nn.Conv2d(64, 64, kernel_size=(3, 3),padding=1)
        self.B12=nn.BatchNorm2d(64)
        self.M12=nn.MaxPool2d(2,2)


        self.C21=nn.Conv2d(64, 128,kernel_size=(3, 3), padding=(1, 1))
        self.B21=nn.BatchNorm2d(128)


        self.C22=nn.Conv2d(128, 128, kernel_size=(3, 3), padding=(1, 1))
        self.B22=nn.BatchNorm2d(128)
        self.M22=nn.MaxPool2d(2,2,padding=1)

        self.C31=nn.Conv2d(128, 256, kernel_size=(3, 3), padding=(1, 1))
        self.B31=nn.BatchNorm2d(256)


        self.C32=nn.Conv2d(256, 256, kernel_size=(3, 3),  padding=(1, 1))
        self.B32=nn.BatchNorm2d(256)


        self.C33=nn.Conv2d(256, 256, kernel_size=(3, 3),  padding=(1, 1))
        self.B33=nn.BatchNorm2d(256)
        self.M33=nn.MaxPool2d(2,2,padding=1)

        self.C41=nn.Conv2d(256, 512, kernel_size=(3, 3),  padding=(1, 1))
        self.B41=nn.BatchNorm2d(512)


        self.C42=nn.Conv2d(512, 512, kernel_size=(3, 3),  padding=(1, 1))
        self.B42=nn.BatchNorm2d(512)


        self.C43=nn.Conv2d(512, 512, kernel_size=(3, 3),  padding=(1, 1))
        self.B43=nn.BatchNorm2d(512)
        self.M43=nn.MaxPool2d(2,2,padding=1)

        self.C51=nn.Conv2d(512, 512, kernel_size=(3, 3), padding=(1, 1))
        self.B51=nn.BatchNorm2d(512)

        self.C52=nn.Conv2d(512, 512, kernel_size=(3, 3),  padding=(1, 1))
        self.B52=nn.BatchNorm2d(512)

        self.C53=nn.Conv2d(512, 512, kernel_size=(3, 3), padding=(1, 1))
        self.B53=nn.BatchNorm2d(512)
        self.M53=nn.MaxPool2d(2,2,padding=1)

        self.FF=nn.Flatten()
        self.L1=nn.Linear(in_features=2048, out_features=4096, bias=True)
        self.D1=nn.Dropout(p=0.5, inplace=False)
        self.L2=nn.Linear(in_features=4096, out_features=4096, bias=True)
        self.D2=nn.Dropout(p=0.5, inplace=False)
        self.L3=nn.Linear(in_features=4096, out_features=10, bias=True)


    def forward(self, xb,G):
       # xb = xb.view(xb.size(0), -1)
        SIGMOID=torch.nn.Sigmoid()
        beta=10
        i=0
        out = self.C11(xb)
        out=self.B11(out)
        out = out*G[i].detach()
        i=i+1



        out = self.C12(out)
        out=self.B12(out)
        out = out*G[i].detach()
        i=i+1
        out = self.M12(out)

        out = self.C21(out)
        out=self.B21(out)
        out = out*G[i].detach()
        i=i+1



        out = self.C22(out)
        out=self.B22(out)
        out = out*G[i].detach()
        i=i+1
        out = self.M22(out)

        out = self.C31(out)
        out=self.B31(out)
        out = out*G[i].detach()
        i=i+1

        out = self.C32(out)
        out=self.B32(out)
        out = out*G[i].detach()
        i=i+1

        out = self.C33(out)
        out=self.B33(out)
        out = out*G[i].detach()
        i=i+1
        out = self.M33(out)

        out = self.C41(out)
        out=self.B41(out)
        out = out*G[i].detach()
        i=i+1


        out = self.C42(out)
        out=self.B42(out)
        out = out*G[i].detach()
        i=i+1


        out = self.C43(out)
        out=self.B43(out)
        out = out*G[i].detach()
        i=i+1
        out = self.M43(out)

        out = self.C51(out)
        out=self.B51(out)
        out = out*G[i].detach()
        i=i+1

        out = self.C52(out)
        out=self.B52(out)
        out = out*G[i].detach()
        i=i+1

        out = self.C53(out)
        out=self.B53(out)
        out = out*G[i].detach()
        i=i+1
        out = self.M53(out)

        out=self.FF(out)
        out=self.L1(out)
        out=self.D1(out)
        out=self.L2(out)
        out=self.D2(out)
        out = self.L3(out)
        return out

In [ ]:
Model_NPF=VGGNEW()

if train_on_gpu:
  Model_NPF.cuda()

# Model_NPV=VGGWeight()

# if train_on_gpu:
#   Model_NPV.cuda()



# for name, param in Model_NPF.named_parameters():
#   print(name)
# print(Model_NPV.C11.weight.lr)

In [ ]:
fisher_dict = {}
optpar_dict = {}
ewc_lambda = 0.4
model = VGGWeight().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
if train_on_gpu:
  model.cuda()

In [ ]:

def test(model, device, x_test, t_test):
    # model.eval()
    test_loss = 0
    correct = 0
    for start in range(0, len(t_test)-1, 256):
      end = start + 256
      with torch.no_grad():
        x, y = torch.from_numpy(x_test[start:end]), torch.from_numpy(t_test[start:end]).long()
        x, y = x.to(device), y.to(device)
        batch_images, labels = x.cuda(), y.cuda()
        All_ones=torch.ones(batch_images.shape).cuda()
        Model_NPF.forward(batch_images)
        output=model.forward(All_ones,Model_NPF.Return_NPF_Gates())
        # output = model(x)
        test_loss += F.cross_entropy(output, y).item()
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(y.view_as(pred)).sum().item()
    test_loss /= len(t_test)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(t_test),
        100. * correct / len(t_test)))
    return 100. * correct / len(t_test)

In [ ]:
def on_task_update(task_id, x_mem, t_mem):

  # model.train()
  optimizer.zero_grad()

  # accumulating gradients
  for start in range(0, len(t_mem)-1, 256):
      end = start + 256
      x, y = torch.from_numpy(x_mem[start:end]), torch.from_numpy(t_mem[start:end]).long()
      x, y = x.to(device), y.to(device)
      batch_images, labels = x.cuda(), y.cuda()
      All_ones=torch.ones(batch_images.shape).cuda()
      Model_NPF.forward(batch_images)
      output=model.forward(All_ones,Model_NPF.Return_NPF_Gates())

      # output = model(x)
      loss = F.cross_entropy(output, y)
      loss.backward()

  fisher_dict[task_id] = {}
  optpar_dict[task_id] = {}

  # gradients accumulated can be used to calculate fisher
  for name, param in model.named_parameters():

    optpar_dict[task_id][name] = param.data.clone()
    fisher_dict[task_id][name] = param.grad.data.clone().pow(2)

In [ ]:
"""EWC"""


def train_ewc(model, device, task_id, x_train, t_train, optimizer, epoch):
    # model.train()

    for start in range(0, len(t_train)-1, 256):
      end = start + 256
      x, y = torch.from_numpy(x_train[start:end]), torch.from_numpy(t_train[start:end]).long()
      x, y = x.to(device), y.to(device)

      optimizer.zero_grad()
      batch_images, labels = x.cuda(), y.cuda()
      All_ones=torch.ones(batch_images.shape).cuda()
      Model_NPF.forward(batch_images)
      output=model.forward(All_ones,Model_NPF.Return_NPF_Gates())

      # output = model(x)
      loss = F.cross_entropy(output, y)

      for task in range(task_id):
        for name, param in model.named_parameters():
          fisher = fisher_dict[task][name]
          optpar = optpar_dict[task][name]
          loss += (fisher * (optpar - param).pow(2)).sum() * ewc_lambda

      loss.backward()
      optimizer.step()

    print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, loss.item()))

In [ ]:
ewc_accs = []
for id, task in enumerate(tasks):
  avg_acc = 0
  print("Training on task-",id)
  (x_train, t_train), _ = task
  for epoch in range(1, 10):
    train_ewc(model, device, id, x_train, t_train, optimizer, epoch)
  on_task_update(id, x_train, t_train)
  for id_test, task in enumerate(tasks):
    print("Testing on task-",id_test)
    _, (x_test, t_test) = task
    acc = test(model, device, x_test, t_test)
    avg_acc = avg_acc + acc
  print("Avg acc: ", avg_acc / 5)
  ewc_accs.append(avg_acc / 5)

Training on task- 0
Train Epoch: 1 	Loss: 0.194600
Train Epoch: 2 	Loss: 0.110539
Train Epoch: 3 	Loss: 0.036785
Train Epoch: 4 	Loss: 0.004565
Train Epoch: 5 	Loss: 0.002601
Train Epoch: 6 	Loss: 0.001360
Train Epoch: 7 	Loss: 0.001427
Train Epoch: 8 	Loss: 0.001023
Train Epoch: 9 	Loss: 0.000750
Testing on task- 0
Test set: Average loss: 0.0002, Accuracy: 9839/10000 (98%)

Testing on task- 1
Test set: Average loss: 0.0110, Accuracy: 1365/10000 (14%)

Testing on task- 2
Test set: Average loss: 0.0104, Accuracy: 1664/10000 (17%)

Testing on task- 3
Test set: Average loss: 0.0111, Accuracy: 1633/10000 (16%)

Testing on task- 4
Test set: Average loss: 0.0102, Accuracy: 1793/10000 (18%)

Avg acc:  32.588
Training on task- 1
Train Epoch: 1 	Loss: 0.235404
Train Epoch: 2 	Loss: 0.045834
Train Epoch: 3 	Loss: 0.004400
Train Epoch: 4 	Loss: 0.002436
Train Epoch: 5 	Loss: 0.001498
Train Epoch: 6 	Loss: 0.000980
Train Epoch: 7 	Loss: 0.000710
Train Epoch: 8 	Loss: 0.000657
Train Epoch: 9 	Loss: